# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

W0811 19:55:57.826459 4360074688 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/ismailkorkmaz/anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0811 19:55:57.895911 4360074688 deprecation_wrapper.py:119] From /Users/ismailkorkmaz/anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [17]:
bob = sy.VirtualWorker(hook, id="bob")

In [22]:
bob._objects

{}

In [10]:
x = th.tensor([1,2,3,4,5])

In [11]:
x = x.send(bob)

In [12]:
bob._objects

{50844634909: tensor([1, 2, 3, 4, 5])}

In [13]:
x.location

<VirtualWorker id:bob #tensors:1>

In [14]:
x.id_at_location

50844634909

In [15]:
x.id

85673375777

Whenever try to perform a command using x, it's going to send a message to self.location 
and say "find the tensor that has this particular ID and execute the command" 

In [16]:
x.owner

<VirtualWorker id:me #tensors:0>

In [17]:
hook.local_worker

<VirtualWorker id:me #tensors:0>

In [18]:
x

(Wrapper)>[PointerTensor | me:85673375777 -> bob:50844634909]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [ ]:
# try this project here!

In [27]:
alice = sy.VirtualWorker(hook, id="alice")

In [28]:
x = th.tensor([1,2,3,4,5])

In [29]:
x_pt = x.send(bob, alice)

In [32]:
x_pt

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:15134434937 -> bob:26537082978]
	-> (Wrapper)>[PointerTensor | me:94609713130 -> alice:81783337398]

In [34]:
x_pt.child.child

{'bob': (Wrapper)>[PointerTensor | me:15134434937 -> bob:26537082978],
 'alice': (Wrapper)>[PointerTensor | me:94609713130 -> alice:81783337398]}

In [35]:
x_pt.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [36]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)

In [38]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [39]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [40]:
x

(Wrapper)>[PointerTensor | me:72102060963 -> bob:48680070475]

In [41]:
y

(Wrapper)>[PointerTensor | me:40638350100 -> bob:40265599356]

In [42]:
z = x + y

In [43]:
z

(Wrapper)>[PointerTensor | me:19102838309 -> bob:35837238001]

In [44]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [45]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:52563036096 -> bob:86936636876]

In [46]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [47]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [48]:
z = (x + y).sum()

In [49]:
z.backward()

(Wrapper)>[PointerTensor | me:13552085083 -> bob:5100336711]

In [50]:
x = x.get()

In [51]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [52]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [ ]:
# try this project here!

In [65]:
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [66]:
input = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True).send(bob)

In [67]:
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True).send(bob)

In [68]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [69]:
for i in range(10):
    pred = input.mm(weights)

    loss = ((pred - target)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [70]:
bob = bob.clear_objects()

In [45]:
bob._objects

{}

In [46]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [47]:
bob._objects

{23831414651: tensor([1, 2, 3, 4, 5])}

In [48]:
del x

In [49]:
bob._objects

{}

In [50]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [51]:
bob._objects

{92212512027: tensor([1, 2, 3, 4, 5])}

In [52]:
x = "asdf"

In [53]:
bob._objects

{}

In [54]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [55]:
x

(Wrapper)>[PointerTensor | me:19384969793 -> bob:49166137090]

In [56]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [57]:
x = "asdf"

In [58]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [59]:
del x

In [60]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [61]:
bob = bob.clear_objects()
bob._objects

{}

In [62]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [63]:
bob._objects

{17426510898: tensor([1, 2, 3, 4, 5])}

In [64]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [65]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

In [130]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [66]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [83]:
from torch import nn, optim
import torch as th

In [84]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [85]:
# A Toy Model
model = nn.Linear(2,1)

In [86]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [87]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(1.4676)
tensor(0.4460)
tensor(0.2613)
tensor(0.1753)
tensor(0.1204)
tensor(0.0835)
tensor(0.0583)
tensor(0.0411)
tensor(0.0292)
tensor(0.0209)
tensor(0.0150)
tensor(0.0109)
tensor(0.0080)
tensor(0.0059)
tensor(0.0043)
tensor(0.0032)
tensor(0.0024)
tensor(0.0018)
tensor(0.0013)
tensor(0.0010)


In [88]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [89]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [90]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [91]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [97]:
train()

tensor(2.8096, requires_grad=True)
tensor(0.0399, requires_grad=True)
tensor(0.0088, requires_grad=True)
tensor(0.0105, requires_grad=True)
tensor(0.0051, requires_grad=True)
tensor(0.0073, requires_grad=True)
tensor(0.0040, requires_grad=True)
tensor(0.0055, requires_grad=True)
tensor(0.0030, requires_grad=True)
tensor(0.0041, requires_grad=True)
tensor(0.0023, requires_grad=True)
tensor(0.0030, requires_grad=True)
tensor(0.0017, requires_grad=True)
tensor(0.0023, requires_grad=True)
tensor(0.0013, requires_grad=True)
tensor(0.0017, requires_grad=True)
tensor(0.0010, requires_grad=True)
tensor(0.0013, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0010, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0002, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [2]:
bob.clear_objects()
alice.clear_objects()

NameError: name 'bob' is not defined

In [124]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [125]:
x = x.send(alice)

In [126]:
bob._objects

{98704031864: tensor([1, 2, 3, 4, 5])}

In [127]:
alice._objects

{7698648012: (Wrapper)>[PointerTensor | alice:7698648012 -> bob:98704031864]}

In [128]:
x

(Wrapper)>[PointerTensor | me:50731323071 -> alice:7698648012]

In [129]:
y = x + x

In [130]:
y

(Wrapper)>[PointerTensor | me:4707667350 -> alice:53205144104]

In [131]:
bob._objects

{98704031864: tensor([1, 2, 3, 4, 5]),
 11905048801: tensor([ 2,  4,  6,  8, 10])}

In [132]:
alice._objects

{7698648012: (Wrapper)>[PointerTensor | alice:7698648012 -> bob:98704031864],
 53205144104: (Wrapper)>[PointerTensor | alice:53205144104 -> bob:11905048801]}

In [133]:
jon = sy.VirtualWorker(hook, id="jon")

In [134]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [135]:
bob._objects

{73267885055: tensor([1, 2, 3, 4, 5])}

In [136]:
alice._objects

{73111579681: (Wrapper)>[PointerTensor | alice:73111579681 -> bob:73267885055]}

In [137]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:73111579681 -> bob:73267885055]

In [138]:
bob._objects

{73267885055: tensor([1, 2, 3, 4, 5])}

In [139]:
alice._objects

{}

In [140]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [141]:
bob._objects

{}

In [142]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [143]:
bob._objects

{35221968256: tensor([1, 2, 3, 4, 5])}

In [144]:
alice._objects

{4319224278: (Wrapper)>[PointerTensor | alice:4319224278 -> bob:35221968256]}

In [145]:
del x

In [146]:
bob._objects

{}

In [147]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [148]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [149]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [150]:
bob._objects

{39331730280: tensor([1, 2, 3, 4, 5])}

In [151]:
alice._objects

{}

In [152]:
x.move(alice)

(Wrapper)>[PointerTensor | me:16694609958 -> alice:16694609958]

In [153]:
bob._objects

{}

In [154]:
alice._objects

{16694609958: tensor([1, 2, 3, 4, 5])}

In [155]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [156]:
bob._objects

{79727459003: tensor([1, 2, 3, 4, 5])}

In [157]:
alice._objects

{16694609958: tensor([1, 2, 3, 4, 5]),
 93673250426: (Wrapper)>[PointerTensor | alice:93673250426 -> bob:79727459003]}

In [158]:
x.remote_get()

(Wrapper)>[PointerTensor | me:12764504866 -> alice:93673250426]

In [159]:
bob._objects

{}

In [160]:
alice._objects

{16694609958: tensor([1, 2, 3, 4, 5]), 93673250426: tensor([1, 2, 3, 4, 5])}

In [161]:
x.move(bob)

(Wrapper)>[PointerTensor | me:12764504866 -> bob:12764504866]

In [162]:
x

(Wrapper)>[PointerTensor | me:12764504866 -> bob:12764504866]

In [163]:
bob._objects

{12764504866: tensor([1, 2, 3, 4, 5])}

In [1]:
alice._objects

NameError: name 'alice' is not defined

# Section Project:
For the final project for this section, you're going to train on the MNIST dataset using federated learning However the gradient should not come up to central server in raw form.

In [6]:
import torch as th
import numpy as np
from torchvision import datasets, transforms
import torchvision.datasets as datasets
from torch.utils.data import Subset
from torch import nn
import torch.nn.functional as F
from torch import optim
import syft as sy
import helper

#Hooking syft to torch
hook = sy.TorchHook(th)

W0813 01:10:03.845244 4371076544 hook.py:97] Torch was already hooked... skipping hooking process


In [ ]:
#mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)

In [7]:
# 10 workers
def create_workers():
    workers = []
    bob = sy.VirtualWorker(hook, id="bob")
    workers.append(bob)
    alice = sy.VirtualWorker(hook, id="alice")
    workers.append(alice)
    andrew = sy.VirtualWorker(hook, id="andrew")
    workers.append(andrew)
    iso = sy.VirtualWorker(hook, id="iso")
    workers.append(iso)
    arthur = sy.VirtualWorker(hook, id="arthur")
    workers.append(arthur)
    ugur = sy.VirtualWorker(hook, id="ugur")
    workers.append(ugur)
    todd = sy.VirtualWorker(hook, id="todd")
    workers.append(todd)
    bojack = sy.VirtualWorker(hook, id="bojack")
    workers.append(bojack)
    tom = sy.VirtualWorker(hook, id="tom")
    workers.append(tom)
    jerry = sy.VirtualWorker(hook, id="jerry")
    workers.append(jerry)
    return workers

In [8]:
def clear_workers(workers):
    for worker in workers:
        worker.clear_objects()

In [9]:
def create_federated_and_test_loaders(workers, trainset, testset):
    federated_train_loader = sy.FederatedDataLoader(
        trainset.federate(workers), batch_size=32, shuffle=True)
    
    test_loader = th.utils.data.DataLoader(testset, batch_size=64, shuffle=False)
    return federated_train_loader, test_loader

In [13]:
def create_train_federated_models(workers, loader, lr=0.12, epoch=5):
    # sends model to first virtual worker
    virtual_model = classifier().send(workers[0])
    optimizer = optim.SGD(virtual_model.parameters(), lr)
    criterion = nn.NLLLoss()
    for n in range(epoch):
        
        i = 0 # Integer to keep up with first index
        
        j = 0 # Integer to keep up with current worker while training
        
        n_mbatch = 0 # Integer to count number of mini-batches per worker
        
        dbLoc = None # Variable to keep up with current worker while looping
        
        cum_loss = 0 # Variable to store cummulative loss
        
        for batch_idx, (imgs, labels) in enumerate(loader):

            # to set dbLoc to the first worker
            if i == 0:
                i = 2
                dbLoc = imgs.location
            
            # to change dbLoc if img is stored on a different worker and also calculate loss
            if dbLoc is not imgs.location:
                print("The total loss for {0} for epoch {2} is {1}".format(workers[j].id, cum_loss/n_mbatch, n+1))
                dbLoc = imgs.location
                j += 1
                
                # Moving the model to a new worker
                virtual_model.move(dbLoc)
                
                # Resetting the cumulative loss and batch count to zero for new worker
                cum_loss = 0
                n_mbatch = 0
                
            optimizer.zero_grad()
            output = virtual_model.forward(imgs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            cum_loss += loss.get().item()
            n_mbatch += 1
        print("The total loss for {0} is {1}".format(workers[j].id, cum_loss/n_mbatch))
        
        # Moving the model to the first worker if training would occur again
        if (n < (epoch - 1)):
            virtual_model.move(workers[0])
    return virtual_model

In [14]:
# To return the model to the central machine
def create_central_model(model):
    return model.get()

In [15]:
# to analyze the private database with the trained model
def analyze_model(model, loader):
    print("Running on", "cpu")
    cum_perc = 0
    for imgs, labels in loader:
        with th.no_grad():
            ps = th.exp(model.forward(imgs))
        top_p, top_class = ps.topk(1, dim=1)
        prob = top_class == labels.view(*top_class.shape)
        prob = prob.float()
        cum_perc += prob.mean().float()
    print("The accuracy of the model is {0}%".format((cum_perc/len(loader))*100))

In [16]:
# Creating models
class classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 10)
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.log_softmax(self.fc5(x), dim=1)
        return x

In [18]:
# Normalization of MNIST data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,))])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [19]:
workers = create_workers()
clear_workers(workers)
federated_loader, test_loader = create_federated_and_test_loaders(workers, 
                                                                  mnist_trainset, mnist_testset)

In [20]:
virtual_model = create_train_federated_models(workers, federated_loader, epoch=2)

The total loss for bob for epoch 1 is 1.8292461243715692
The total loss for alice for epoch 1 is 0.8177220658101934
The total loss for andrew for epoch 1 is 0.5765938726352885
The total loss for iso for epoch 1 is 0.4157841718418801
The total loss for arthur for epoch 1 is 0.35828813367543066
The total loss for ugur for epoch 1 is 0.41314220543395963
The total loss for todd for epoch 1 is 0.30566066761124644
The total loss for bojack for epoch 1 is 0.3291913891171521
The total loss for tom for epoch 1 is 0.27036349895469686
The total loss for jerry is 0.18167539644352299
The total loss for bob for epoch 2 is 0.21616332936397892
The total loss for alice for epoch 2 is 0.2246444937079511
The total loss for andrew for epoch 2 is 0.21427305173525152
The total loss for iso for epoch 2 is 0.18571538218554665
The total loss for arthur for epoch 2 is 0.18187832959154818
The total loss for ugur for epoch 2 is 0.17347133345901966
The total loss for todd for epoch 2 is 0.17373526056713245
The tot

In [21]:
central_model = create_central_model(virtual_model)

In [22]:
analyze_model(central_model, test_loader)

Running on cpu
The accuracy of the model is 92.8841552734375%
